In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

from flask import Flask, jsonify
import pandas as pd

In [2]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

Base = automap_base()

Base.prepare(engine, reflect=True)

measurement = Base.classes.measurement

station = Base.classes.station

session = Session(engine)

In [3]:
dates=[]
for i in session.query(measurement).all():
    dates.append((i.__dict__['date']))


prcp =[]
for i in session.query(measurement).all():
    prcp.append((i.__dict__['prcp']))


precipitation = {k: v for k, v in zip(dates, prcp)}

In [4]:
query_stations= session.query(measurement.station).distinct().all()
query_stations

[('USC00519397'),
 ('USC00513117'),
 ('USC00514830'),
 ('USC00517948'),
 ('USC00518838'),
 ('USC00519523'),
 ('USC00519281'),
 ('USC00511918'),
 ('USC00516128')]

In [5]:
active_stations=session.query(measurement.station,
    func.count(measurement.id).label('qty')
    ).group_by(measurement.station
    ).order_by(desc('qty')).all()

active_stations[0][0]

'USC00519281'

In [6]:
print(f'The most active station was: {active_stations[0][0]}')

The most active station was: USC00519281


In [7]:
import datetime as dt
from dateutil.relativedelta import *

first_date = (session.query(measurement.date)
              .order_by(measurement.date.desc()).first())

query_date =first_date[0].split("-")

for i in range(0, len(query_date)): 
    query_date[i] = int(query_date[i])

query_date=(dt.date(query_date[0],query_date[1],query_date[2]))


query_date = query_date+relativedelta(months=-12)

active_temp = session.query(measurement.tobs).\
    filter(measurement.date > query_date).\
    filter(measurement.station == active_stations[0][0]).all()

flat_list = []
for sublist in active_temp:
    for item in sublist:
        flat_list.append(item)
active_temp = flat_list

tobs_dict = {k: v for k, v in zip(dates, active_temp)}
print(type(query_date))

<class 'datetime.date'>


In [8]:
first_date = (session.query(measurement.date)
              .order_by(measurement.date.desc()).first())
first_date[0]

'2017-08-23'

In [9]:
not_date = '2016-08-23'

In [10]:
def start(given_date):
    
    given_date =given_date.split("-")

    for i in range(0, len(given_date)): 
        given_date[i] = int(given_date[i])
        
    given_date=(dt.date(given_date[0],given_date[1],given_date[2]))
        
    high_temp= session.query(func.max(measurement.tobs).\
    filter(measurement.date > given_date)).all()
    
    low_temp= session.query(func.min(measurement.tobs).\
    filter(measurement.station == active_stations[0][0])).all()
    
    mean_temp= session.query(func.avg(measurement.tobs).\
    filter(measurement.station == active_stations[0][0])).all()
    


In [11]:
start(not_date)

In [12]:
def start(given_date):
    
    given_date =given_date.split("-")

    for i in range(0, len(given_date)): 
        given_date[i] = int(given_date[i])
        
    given_date=(dt.date(given_date[0],given_date[1],given_date[2]))
        
    max_temp = session.query(func.max(measurement.tobs).\
    filter(measurement.date >= given_date)).all()
                             
    min_temp = session.query(func.min(measurement.tobs).\
    filter(measurement.date >= given_date)).all()
                            
    avg_temp= session.query(func.avg(measurement.tobs).\
    filter(measurement.date >= given_date)).all()
    
    start_df  = pd.DataFrame({
    "Start Date": given_date,
    "Lowest Temperature" : min_temp[0],
    "Highest Temperature": max_temp[0],
    "Average Temperature": round(avg_temp[0][0],2)
    })
    
    return (f"The start date is {given_date}\n"
               f"the minimum temperature since then is {min_temp[0][0]} F \n"
               f"the average temperature {round(avg_temp[0][0],2)} F"
               f", and the max temperature {max_temp[0][0]} F")
#     return print(results)

In [13]:
start('2017-05-01')

'The start date is 2017-05-01\nthe minimum temperature since then is 65.0 F \nthe average temperature 77.27 F, and the max temperature 87.0 F'

In [19]:
def fer(start_date,end_date):

    start_date =start_date.split("-")

    for i in range(0, len(start_date)): 
        start_date[i] = int(start_date[i])
        
    start_date=(dt.date(start_date[0],start_date[1],start_date[2]))
    
    end_date =end_date.split("-")

    for i in range(0, len(end_date)): 
        end_date[i] = int(end_date[i])
        
    end_date=(dt.date(end_date[0],end_date[1],end_date[2]))
    
    max_temp = session.query(func.max(measurement.tobs).\
    filter(measurement.date >= start_date).\
    filter(measurement.date <= end_date)).\
    all()
                             
    min_temp = session.query(func.min(measurement.tobs).\
    filter(measurement.date >= start_date).\
    filter(measurement.date <= end_date)).\
    all()
                            
    avg_temp= session.query(func.avg(measurement.tobs).\
    filter(measurement.date >= start_date).\
    filter(measurement.date <= end_date)).\
    all()
    
#     session.close()

    return (f"The start date is {start_date}\n"
            f" the end date is {end_date}\n"
               f"the minimum temperature during that period is {min_temp[0][0]} F \n"
               f"the average temperature {round(avg_temp[0][0],2)} F"
               f", and the max temperature {max_temp[0][0]} F")
    

In [20]:
fer('2013-05-01','2014-05-01')

'The start date is 2013-05-01\n the end date is 2014-05-01\nthe minimum temperature during that period is 57.0 F \nthe average temperature 72.84 F, and the max temperature 86.0 F'

In [ ]:
start_date= "2013-05-01"

In [ ]:
start_date =start_date.split("-")

In [ ]:
for i in range(0, len(start_date)): 
    start_date[i] = int(start_date[i])

In [ ]:
given_date = not_date
for i in range(0, len(given_date)): 
    given_date[i] = int(given_date[i])